In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import sys

import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import load
from pathlib import Path

base_path = f'{Path.home()}/SageMaker'
print(base_path)
%ls $base_path

/home/ec2-user/SageMaker
3DDFA_V2/          lost+found/                setup.ipynb
autostop.py        mean_landmarks.ipynb       stylegan2_pytorch/
autostop.py.1      model.py                   train_data/
cron-shutdown.sh*  on-start.sh*               train.py
custom-miniconda/  photometric_optimization/  train_v2.py
data/              pretrained_models/         utils.ipynb
Face_Seg/          __pycache__/


In [4]:
import pprint

pprint.pprint(sys.path)

['',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/my_pytorch17_py36/lib/python36.zip',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/my_pytorch17_py36/lib/python3.6',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/my_pytorch17_py36/lib/python3.6/lib-dynload',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/my_pytorch17_py36/lib/python3.6/site-packages',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/my_pytorch17_py36/lib/python3.6/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython']


## Helper funcs

In [5]:
def normalize_float_img(x):
#     if len(x.size()) > 3:
#         x = x.squeeze()
    return x.mul_(127.5/255.).add_(0.5)


def convert_to_uint8(images):
    # Add 0.5 after unnormalizing to [0, 255] to round to nearest integer
    scale = 255 / 2
    images = images.mul(scale) \
            .add_(0.5 + scale) \
            .clamp(0, 255) \
            .permute(0, 2, 3, 1) \
            .to('cpu', torch.uint8) \
            .squeeze()
    return images

## Define the encoder network

In [ ]:
# class StyleEncoder(nn.Module):
#     def __init__(self, dims_in=(18,512), dims_out=(18,32)):
#         super(StyleEncoder, self).__init__()
        
#         self.dims_in = dims_in
#         self.dims_out = dims_out
        
#         self.layers = [nn.Linear(dims_in[-1], dims_out[-1]) \
#                        for _ in range(0, dims_in[0])]
#         # Using ModuleList so that this layer list can be moved to CUDA                      
#         self.layers = torch.nn.ModuleList(self.layers)
        
#     def forward(self, x):
#         ''' 
#         Pass each style dimension from w+ through the
#         independent linear mapping network
        
#         args:
#             x: w+ used with StyleGAN(2); dims=(18,512)
#         returns:
#             outputs: independent linear mapping to lower dimensional
#                      space; dims=(18,32)
#         '''
#         outputs = []
#         for style, layer in zip(x, self.layers):
#             outputs.append(layer(style).unsqueeze(0))
        
#         outputs = torch.cat(outputs, dim=0)
#         return outputs
        
        

In [ ]:
# enc = StyleEncoder()
# enc = enc.cuda()

In [ ]:
# x = torch.randn((3, 18, 512))
# print(x.shape)

# out = enc(x.cuda())
# print(out.shape)

## Define decoder
- concatenates each output dimension with control parameters, and similar to the encoder, independently linearly decodes each dimension back to dimensions of w+

In [ ]:
# class StyleDecoder(nn.Module):
#     def __init__(self, dims_in=(18, 268), dims_out=(18, 512)):
#         super(StyleDecoder, self).__init__()
        
#         self.dims_in = dims_in
#         self.dims_out = dims_out
        
#         ### TODO:
#         ### - Update this to produce the correct results based on dimensions
#         ###   of input and output.
#         ###
#         self.layers = [nn.Linear(dims_in[-1], dims_out[-1]) \
#                        for _ in range(0, dims_in[0])]
        
#         # Using ModuleList so that this layer list can be moved to CUDA
#         #
#         self.layers = torch.nn.ModuleList(self.layers)
        
#     ### TODO
#     ### - Finish me
#     def forward(self, w_enc, params):
#         ''' 
#         Concatenate x and p and then decode.
        
#         Args:
#             w_enc: Encoded latent
#             params: 3DMM params
        
#         Returns:
#             Decoded output
#         ''' 
#         # Flatten lighting parameter to match dimensions of other parameters in list. 
#         #
#         batch_size = params[0].shape[0]
#         params = list(params)
#         params[-1] = params[-1].view(batch_size, -1) # [B,9,3] -> [B,27]
        
        
#         # Concatenate all parameters
#         #
#         params_flat = torch.cat(params, dim=-1)
#         style_dims = self.dims_in[0] # 18
#         params_flat = params_flat.unsqueeze(1).expand(-1, style_dims, -1) 

        
#         # This result is fed into the decoder.
#         #
#         outputs = []
#         x = torch.cat((w_enc, params_flat), dim=-1)
         
    
#         # Treat each "style_dim" independently, therefore pass each
#         # through an independent linear layer.
#         #
#         for idx, layer in enumerate(self.layers):
#             out = layer(x[:,idx,:])
#             out = out.unsqueeze(1) # [B,512] -> [B,1,512]
#             outputs.append(out)
    
    
#         outputs = torch.cat(outputs, dim=1) # [B,18,512]
#         return outputs

## Instantiate models 
- DFR regressor
- Generator (StyleGAN2)

In [ ]:
# device = 'cuda'
# dfr = load.dfr(base_path, device)
# g_ema = load.generator(base_path, device)

## Steps
1) Regress render parameters **p** i.e. $F(\textbf{w}_{+}) = \textbf{p}$ <br>
2) Pass $\textbf{w}_{+}$ through encoder, producing lower dimensional vector **l** <br>
3) Concatenate **l** with **p** and pass through decoder to produce vector **d** (18,512) <br>
4) Add $\textbf{w}_{+}$ and **d** to produce final latent $\hat{w}$


In [ ]:
# batch_size = 3
# with torch.no_grad():
# #     images = []
#     batch_latents = g_ema.get_latent(torch.randn((batch_size,
#                                                   18,
#                                                   512)).to(device)) #.to('cpu')

# #     imgs_gen, _ = g_ema([batch_latents.to(device)], **kwargs)
# #     imgs_gen = imgs_gen.cpu()
# #     images.append(normalize_float_img(imgs_gen))

### 1)

In [ ]:
# # shape, expression, pose, tex, cam, lights = dfr(batch_latents.view(3,-1))
# # print(shape.shape)
# # print(expression.shape)
# # print(pose.shape)
# # print(tex.shape)
# # print(cam.shape)
# # print(lights.shape)

# p = dfr(batch_latents.view(batch_size,-1))

# # for param in p:
# #     print(param.shape)

### 2)

In [ ]:
# l = enc(batch_latents)
# # print(l.shape)

## 3)

In [ ]:
# dec = StyleDecoder().cuda()

In [ ]:
# # p = dfr(batch_latents.view(batch_size,-1))
# # l = enc(batch_latents)
# out = dec(l, p)

In [ ]:
# for x in out:
#     print(x.shape)

## Create RigNet
- Encapsulates encoder and decoder

In [ ]:
# class RigNet(nn.Module):
#     def __init__(self,
#             dim_enc_in=(18,512),
#             dim_enc_out=(18,32),
#             dim_dec_in=(18, 268),
#             dim_dec_out=(18, 512)):
        
#         super(RigNet, self).__init__()
        
#         self.encoder = StyleEncoder(dim_enc_in, dim_enc_out)
#         self.decoder = StyleDecoder(dim_dec_in, dim_dec_out)
        
#     def forward(self, w_plus, params):
#         '''
#         Encode, "mix" w/ 3DMM params, and decode.
        
#         Args:
#             w_plus: Latent from mapping network of StyleGAN(2) generator.
#             params: 3DMM params from DFR.
            
#         Returns:
#             d_plus: Latent from "mixing" 3DMM with latent 'w_plus'
#         '''
#         x = self.encoder(w_plus)
#         x = self.decoder(x, params)
#         d_plus = torch.add(x, w_plus)
        
#         return d_plus
        
        

In [ ]:
# rignet = RigNet().cuda()

### Full working example.

In [ ]:
# from torch.optim import SGD, Adam, Adadelta, Adagrad, Adamax, ASGD, LBFGS, RMSprop, Rprop
# from tqdm.auto import tqdm, trange
# # from tqdm import tqdm_notebook as tqdm

# #optimizer = Adam(model.parameters(), lr=1e-3) # default Adam
# # optimizer = SGD(model.parameters(), lr=.1, momentum=.9) # default SGD
# #optimizer = SGD(model.parameters(), lr=1e-3, momentum=.9) # original SGD
# # optimizer = Adadelta(rignet.parameters(), lr=0.01) # default Adadelta
# #optimizer = Adagrad(model.parameters()) # default Adagrad
# #optimizer = Adamax(model.parameters()) # default Adamax
# #optimizer = ASGD(model.parameters()) # default ASGD
# #optimizer = LBFGS(model.parameters()) # default LBFGS
# #optimizer = RMSprop(model.parameters()) # default RMSprop
# #optimizer = Rprop(model.parameters()) # default Rprop

In [ ]:
# from models.RigNet import RigNet

# rignet = RigNet().cuda()
# dfr = load.dfr(base_path, device)

# optimizer = Adadelta(rignet.parameters(), lr=0.01) # default Adadelta

In [ ]:
# # sample the latent code and pass through mapping network
# # producing 18 dim style code for generator.
# #
# for i in range(0, 5):
#     batch_size = 5
#     z = torch.randn((batch_size, 18, 512)).to(device)
#     with torch.no_grad():
#         w = g_ema.get_latent(z)

#     # 3DMM params from latent
#     #
#     p = dfr(w.view(batch_size, -1))

#     # Generate final latent code for generator "mixed" w/ 3DMM params.
#     #
#     w_hat = rignet(w, p)
#     print(w.shape)
# #     print(p.shape)
#     print(w_hat.shape)

#     # L2-loss
#     #
#     optimizer.zero_grad()
#     loss = (w_hat - w).abs().square().mean()
#     loss.backward()
#     optimizer.step()


## Implement the various RigNet losses
**Reconstruction loss**
- Ensure can faithfully reproduce latent codes in the training corpus
- Want to ensure $RigNet(w,F(w)) = w$
- Use the following $l_{2}-loss$ <br>
$L_{rec} = ||RigNet(w, F(w)) - w ||_{2}^{2}$

In [ ]:
# Generate the training corpus for reconstruction loss
#
batch_size = 1000
z = torch.randn((batch_size, 18, 512)).to(device)
# with torch.no_grad():
#     w_plus = g_ema.get_latent(z)
    
w_plus = g_ema.get_latent(z).detach()

print(w_plus.shape)

In [ ]:
rignet = RigNet().cuda()
dfr = load.dfr(base_path, device)

optimizer = Adam(rignet.parameters(), lr=1e-3) # default Adam # Trains
# optimizer = Adadelta(rignet.parameters(), lr=0.001) # default Adadelta # Does not train??
# optimizer = SGD(rignet.parameters(), lr=.1, momentum=.9) # default SGD # Trains
# optimizer = RMSprop(rignet.parameters()) # default RMSprop # Trains
# optimizer = ASGD(rignet.parameters()) # default ASGD # Does not train??
# optimizer = Adamax(rignet.parameters()) # default Adamax # Trains

In [ ]:
# Create a dataloader to batch up latents in consumable chunks.
#
from torch.utils.data import DataLoader

batch_size = 10
loader = DataLoader(w_plus, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
epochs = 30

# pbar = tqdm(range(0, epochs), dynamic_ncols=True, smoothing=0.01)
pbar = trange(epochs, dynamic_ncols=True, smoothing=0.01)
for epoch in pbar:
#     print("epoch: ", epoch)
    for step, w in enumerate(loader):
#         print(w.shape)
        p = dfr(w.view(batch_size, -1))
#         print(p.shape)
        w_hat = rignet(w, p)

        # L2-loss
        #
        optimizer.zero_grad()
        loss = (w_hat - w).abs().square().mean()
        loss.backward()
        optimizer.step()

        pbar.set_description
        (
            (
                f"total: {loss:.4f}; recon: {loss:.4f}; "
            )
        )
        
    print(loss.item())


---
---

#### Cycle-consistent editing loss
1. Create small dataset in memory to test
    - latents **w** and **v**
    - corresponding pose parameters **$p_{w}$** and **$p_{v}$**
    - corresponding images **$I_{w}$** and **$I_{v}$**
    - landmarks, masks, etc.

In [6]:
from torch.optim import SGD, Adam, Adadelta, Adagrad, Adamax, ASGD, LBFGS, RMSprop, Rprop
from tqdm.auto import tqdm, trange
# from tqdm import tqdm_notebook as tqdm

#optimizer = Adam(model.parameters(), lr=1e-3) # default Adam
# optimizer = SGD(model.parameters(), lr=.1, momentum=.9) # default SGD
#optimizer = SGD(model.parameters(), lr=1e-3, momentum=.9) # original SGD
# optimizer = Adadelta(rignet.parameters(), lr=0.01) # default Adadelta
#optimizer = Adagrad(model.parameters()) # default Adagrad
#optimizer = Adamax(model.parameters()) # default Adamax
#optimizer = ASGD(model.parameters()) # default ASGD
#optimizer = LBFGS(model.parameters()) # default LBFGS
#optimizer = RMSprop(model.parameters()) # default RMSprop
#optimizer = Rprop(model.parameters()) # default Rprop

In [7]:
device = 'cuda'

# rignet = load.rignet(base_path).cuda()
dfr = load.dfr(base_path).cuda()

Output params dims:  236


In [8]:
# rignet = load.rignet(base_path, one_hot=True).cuda()
rignet = load.rignet(base_path, one_hot=False).cuda()

Creating RigNet model...


In [11]:
# sys.path.append(f'{base_path}/stylegan2_pytorch')
g_ema = load.generator(base_path).cuda()

path to StyleGAN2 weights:  /home/ec2-user/SageMaker/pretrained_models/stylegan2-ffhq-config-f.pt
Loading weights...  /home/ec2-user/SageMaker/pretrained_models/stylegan2-ffhq-config-f.pt


In [12]:
import yaml

with open(f'{base_path}/photometric_optimization/config/config.yaml') as f:
    config = yaml.safe_load(f)
    
    
model_params = config['model_params']
path_pretrained = f'{base_path}/pretrained_models'
model_params['flame_model_path'] = f'{path_pretrained}/generic_model.pkl'
model_params['flame_lmk_embedding_path'] = f'{path_pretrained}/landmark_embedding.npy'
model_params['tex_space_path'] = f'{path_pretrained}/FLAME_texture.npz'


# del render
render = load.renderer(base_path, model_params).cuda()
flame = load.flame(base_path, model_params).cuda()
flametex = load.flametex(base_path, model_params).cuda()

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/my_pytorch17_py36/lib/python3.6/site-packages/pytorch3d/io/obj_io.py:457: UserWarning: Mtl file does not exist: ./data/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


creating the FLAME Decoder


/home/ec2-user/SageMaker/photometric_optimization/models/FLAME.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('dynamic_lmk_faces_idx', torch.tensor(lmk_embeddings['dynamic_lmk_faces_idx'], dtype=torch.long))
/home/ec2-user/SageMaker/photometric_optimization/models/FLAME.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('dynamic_lmk_bary_coords', torch.tensor(lmk_embeddings['dynamic_lmk_bary_coords'], dtype=self.dtype))


In [11]:
# # Produce latents
# #
# batch_size = 4
# total_size = batch_size * 1  

# w = g_ema.get_latent(torch.randn((total_size, 18, 512)).to(device)).detach()
# v = g_ema.get_latent(torch.randn((total_size, 18, 512)).to(device)).detach()

# # Produce parameters
# #
# ### TODO:
# ### DFR model should not require inputs to be reshaped. Should be
# ### logic inside the model itself.
# ###
# p_w = dfr(w.view(w.shape[0], -1))
# p_v = dfr(v.view(v.shape[0], -1))

# # Generate images
# #
# w_mean = g_ema.mean_latent(4096) # For truncation.
# noise = g_ema.make_noise()


# kwargs = {
#     'truncation_latent': w_mean,
#     'truncation': 0.7,
#     'noise': noise,
#     'randomize_noise': False,
#     'input_is_latent': True,
#     'return_latents': False
# }


# # del I_w
# # del I_v
# I_w, _ = g_ema([w.to(device)], **kwargs)
# I_w = I_w.cpu()
# I_v, _ = g_ema([v.to(device)], **kwargs)
# I_v = I_v.cpu()


# # Normalize and interpolate to size expected by flame and landmark models.
# #

# dims = (224, 224)
# I_w = normalize_float_img(I_w)
# # I_w = torch.cat(I_w, dim=0)
# I_w = F.interpolate(I_w, dims)

# I_v = normalize_float_img(I_v)
# # I_v = torch.cat(I_v, dim=0)
# I_v = F.interpolate(I_v, dims)


In [12]:
# import matplotlib.pyplot as plt

# plt.imshow(convert_to_uint8(I_w[0].unsqueeze(0))); plt.axis('off'); plt.show(); 
# plt.imshow(convert_to_uint8(I_v[0].unsqueeze(0))); plt.axis('off'); plt.show(); 

In [14]:
# For testing, reuse DFR dataset so
# don't need to calculate everything.
#

from torch.utils import data
import glob


class DatasetSiamese(data.Dataset):
    def __init__(self, path_to_dir_A, path_to_dir_B):
        self.path_A = path_to_dir_A
        self.path_B = path_to_dir_B
        self.files_A = glob.glob1(self.path_A, '*.pkl')
        self.files_B = glob.glob1(self.path_B, '*.pkl')
        
    def __len__(self):
        len_A = len(glob.glob1(self.path_A, '*.pkl'))
        len_B = len(glob.glob1(self.path_A, '*.pkl'))
        assert len_A == len_B
        return len_A
    
    def __getitem__(self, index):
        filename = str(index).zfill(6) + '.pkl'
        example_A = torch.load(f'{self.path_A}/{filename}')
        example_B = torch.load(f'{self.path_B}/{filename}')
        return example_A, example_B

In [15]:
train_dir_A = f'{base_path}/train_data/rignet_testing_A'
print("train_dir: ", train_dir_A)
train_dir_B = f'{base_path}/train_data/rignet_testing_B'
print("train_dir: ", train_dir_B)
dataset = DatasetSiamese(train_dir_A, train_dir_B)

batch_size = 4
sampler = None
# # if args.distributed:
# #     sampler = torch.utils.data.distributed.DistributedSampler(
# #         dataset,
# #         num_replicas=torch.cuda.device_count(),
# # # #             num_replicas=4,
# # #             rank = args.local_rank,
# #     )

loader = data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=(sampler is None),
    num_workers=0,
#     collate_fn=collate_fn,
    pin_memory=True,
    sampler=sampler,
    drop_last=True
)

### TODO:
### - Does not work. Looks for size() in Dataset as if it's a tensor.
###   How to update to properly work?
###
# dataset = torch.utils.data.TensorDataset(dataset_A, dataset_B)
# dataloader = data.DataLoader(dataset, batch_size=4, shuffle=True)
# # for index, (xb1, xb2) in enumerate(dataloader):


# final_dataset = torch.utils.data.ConcatDataset((dataset_A, dataset_B))
# loader = data.DataLoader(
#     dataset=final_dataset,
#     batch_size=batch_size,
#     shuffle=(sampler is None),
#     num_workers=0,
#     pin_memory=True,
#     sampler=sampler,
#     drop_last=True
# )


train_dir:  /home/ec2-user/SageMaker/train_data/rignet_testing_A
train_dir:  /home/ec2-user/SageMaker/train_data/rignet_testing_B


In [17]:
# for A, B in loader:
#     latents = A['latents'].cuda()
#     landmarks_2d_gt = A['landmarks_2d_gt'].cuda()
#     images = A['images'].cuda()
#     image_masks = A['image_masks'].cuda()

#     print(A.keys())
#     print(B.keys())

In [18]:
for A, B in loader:
    w_A = A['latents'].cuda()
    landmarks_2d_gt_A = A['landmarks_2d_gt'].cuda()
    images_A = A['images'].cuda()
    image_masks_A = A['image_masks'].cuda()

    w_B = B['latents'].cuda()
    landmarks_2d_gt_B = B['landmarks_2d_gt'].cuda()
    images_B = B['images'].cuda()
    image_masks_B = B['image_masks'].cuda()
#     print(A.keys())
#     print(B.keys())
    break

#### One-hot conditional labels for supporting RigNet model.

In [36]:
onehot_pose = F.one_hot(torch.tensor([4]), 6).unsqueeze(0)
onehot_expr = F.one_hot(torch.tensor([2]), 6).unsqueeze(0)

# labels_in = F.one_hot(torch.tensor([4]), 6).unsqueeze(0).expand(4,-1,-1)
# labels_in = labels_in.cuda()

# labels_in = torch.cat((onehot_pose, onehot_expr), dim=0).cuda()

labels_in = onehot_pose

labels_in = labels_in.expand(batch_size, -1, -1)
labels_in = labels_in.cuda()

print(labels_in.shape)
print(onehot_pose.shape)
print("labels_in: ", labels_in)
print("labels_pose: ", onehot_pose)
print("labels_expr: ", onehot_expr)
print(onehot_pose + onehot_expr)


torch.Size([4, 1, 6])
torch.Size([1, 1, 6])
labels_in:  tensor([[[0, 0, 0, 0, 1, 0]],

        [[0, 0, 0, 0, 1, 0]],

        [[0, 0, 0, 0, 1, 0]],

        [[0, 0, 0, 0, 1, 0]]], device='cuda:0')
labels_pose:  tensor([[[0, 0, 0, 0, 1, 0]]])
labels_expr:  tensor([[[0, 0, 1, 0, 0, 0]]])
tensor([[[0, 0, 1, 0, 1, 0]]])


In [21]:
# 1) Calculate parameters from latents
#
# params_A = dfr(w_A.view(w_A.shape[0], -1))
# params_B = dfr(w_B.view(w_B.shape[0], -1))

params_A = dfr(nn.Flatten()(w_A))
params_B = dfr(nn.Flatten()(w_B))

In [37]:
# 2) Transfer semantic "style" (e.g. pose) from one latent to another.
#
w_Ahat = rignet(w_B, params_A, labels_in) # Transfer semantic params from A to latent B
w_Bhat = rignet(w_A, params_B, labels_in) # Transfer semantic params from B to latent A

In [38]:
# 3) Create the cycle, where transferred paramter to latent, and latent to parameter
#    should match.
#
params_AB = dfr(nn.Flatten()(w_Ahat))
params_BA = dfr(nn.Flatten()(w_Bhat))

# shape, expression, pose, tex, cam, lights = dfr(latents.view(args.batch_size, -1))
params_edit_A = [x.clone() for x in params_A]
params_edit_A[2] = params_AB[2] # Ensure pose is maintained

params_edit_B = [x.clone() for x in params_B]
params_edit_B[2] = params_BA[2] # Ensure pose is maintained


#### Perform the rendering and return loss

In [16]:
def cycle_editing_loss(example, param, _flame, _flametex):
    import util
    
    loss_mse = nn.MSELoss().cuda()
    
#     w_A = A['latents'].cuda()
#     landmarks_2d_gt_A = A['landmarks_2d_gt'].cuda()
#     images_A = A['images'].cuda()
#     image_masks_A = A['image_masks'].cuda()

#     w_B = B['latents'].cuda()
#     landmarks_2d_gt_B = B['landmarks_2d_gt'].cuda()
#     images_B = B['images'].cuda()
#     image_masks_B = B['image_masks'].cuda()

    latents = example['latents'].cuda()
    landmarks_2d_gt = example['landmarks_2d_gt'].cuda()
    landmarks_3d_gt = example['landmarks_3d_gt'].cuda()
    images = example['images'].cuda()
    image_masks = example['image_masks'].cuda()
    
    
    # shape, expression, pose, tex, cam, lights = dfr(latents.view(args.batch_size, -1))
    shape, expression, pose, tex, cam, lights = param
    vertices, landmarks2d, landmarks3d = flame(shape_params=shape,
                                               expression_params=expression,
                                               pose_params=pose)

    # render
    #
    albedos = flametex(tex) / 255.
    
    trans_vertices = util.batch_orth_proj(vertices, cam);
    trans_vertices[..., 1:] = - trans_vertices[..., 1:]
    landmarks2d = util.batch_orth_proj(landmarks2d, cam);
    landmarks2d[..., 1:] = - landmarks2d[..., 1:]
    landmarks3d = util.batch_orth_proj(landmarks3d, cam);
    landmarks3d[..., 1:] = - landmarks3d[..., 1:]
    
    losses = {}
    losses['landmark_2d'] = util.l2_distance(landmarks2d[:, :, :2],
                                              landmarks_2d_gt[:, :, :2]) * 1.0
    losses['landmark_3d'] = util.l2_distance(landmarks3d[:, :, :2],
                                              landmarks_3d_gt[:, :, :2]) * 1.0
    losses['shape_reg'] = (torch.sum(shape ** 2) / 2) * 1e-4
    losses['expression_reg'] = (torch.sum(expression ** 2) / 2) * 1e-4
    losses['pose_reg'] = (torch.sum(pose ** 2) / 2) * 1e-4 #config.w_pose_reg

#     # Regularize learned texture.
#     #
#     losses['texture_reg'] = loss_mse(albedos, texture_mean.repeat(args.batch_size, 1, 1, 1)) 
    
    
    ## render
    albedos = flametex(tex) / 255.
    ops = render(vertices, trans_vertices, albedos, lights)

    predicted_images = ops['images']
    losses['photometric_texture'] = (image_masks * (predicted_images - images).abs()).mean() \
                                    * 1.0 #config.w_pho


    all_loss = 0.
    for key in losses.keys():
        all_loss = all_loss + losses[key]
#         losses_to_plot[key].append(losses[key].item()) # Store for plotting later.


    losses['all_loss'] = all_loss
#     losses_to_plot['all_loss'].append(losses['all_loss'].item())

    return losses


In [40]:
loss_A = cycle_editing_loss(A, params_edit_A, flame, flametex)
loss_B = cycle_editing_loss(B, params_edit_B, flame, flametex)

In [41]:
optim = torch.optim.Adam(
    rignet.parameters(),
    lr=1e-3,
    weight_decay=0.0001 # config.e_wd
)

In [42]:
total_loss = loss_A['all_loss'] + loss_B['all_loss']

optim.zero_grad()
total_loss.backward()
optim.step()

In [46]:
total_loss

tensor(2.2299, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)

---

## Train

In [27]:

del rignet

# one_hot = True
one_hot = False
rignet = load.rignet(base_path, one_hot=one_hot).cuda()

Creating RigNet model...


In [29]:
# optim = torch.optim.Adam(
#     rignet.parameters(),
#     lr=1e-3,    
# #     weight_decay=0.00001 # config.e_wd
# )

optim = Adamax(rignet.parameters()) # default Adamax # Trains

In [ ]:

epochs = int(10e3)


# shape, expression, pose, tex, cam, lights = dfr(latents.view(args.batch_size, -1))
# onehot_cam = F.one_hot(torch.tensor([4]), 6).unsqueeze(0)
onehot_pose = F.one_hot(torch.tensor([2]), 6).unsqueeze(0)


# labels_in = torch.cat((onehot_pose, onehot_expr), dim=0).cuda()
labels_in = onehot_pose
labels_in = labels_in.expand(batch_size, -1, -1)
labels_in = labels_in.cuda()


for epoch in range(0, epochs):
    for A, B in loader:
        w_A = A['latents'].cuda()
        landmarks_2d_gt_A = A['landmarks_2d_gt'].cuda()
        images_A = A['images'].cuda()
        image_masks_A = A['image_masks'].cuda()

        w_B = B['latents'].cuda()
        landmarks_2d_gt_B = B['landmarks_2d_gt'].cuda()
        images_B = B['images'].cuda()
        image_masks_B = B['image_masks'].cuda()


        # 1) Calculate parameters from latents
        #
        # params_A = dfr(w_A.view(w_A.shape[0], -1))
        # params_B = dfr(w_B.view(w_B.shape[0], -1))
        params_A = dfr(nn.Flatten()(w_A))
        params_B = dfr(nn.Flatten()(w_B))
        
        
        # 2) Transfer semantic "style" (e.g. pose) from one latent to another.
        #
        w_Ahat = rignet(w_B, params_A, labels_in) # Transfer semantic params from A to latent B
        w_Bhat = rignet(w_A, params_B, labels_in) # Transfer semantic params from B to latent A

        
        # 3) Create the cycle, where transferred paramter to latent, and latent to parameter
        #    should match.
        #
        params_AB = dfr(nn.Flatten()(w_Ahat))
        params_BA = dfr(nn.Flatten()(w_Bhat))

        
        # shape, expression, pose, tex, cam, lights = dfr(latents.view(args.batch_size, -1))
        params_edit_A = [x.clone() for x in params_A]
        params_edit_A[2] = params_AB[2] # Ensure pose is maintained
        params_edit_B = [x.clone() for x in params_B]
        params_edit_B[2] = params_BA[2] # Ensure pose is maintained


        # Losses
        # ------------------------------------------------
        total_loss = 0.0
        
        
        # Latent reconstruction loss.
        #
        scale_recon = 0.1
        loss_recon = 0.0
        w_hat = rignet(w_A, params_A, labels_in) # Reconsruct self
        loss_recon = (w_hat - w_A).abs().square().mean() * scale_recon
        w_hat = rignet(w_B, params_B, labels_in) # Reconsruct self
        loss_recon += (w_hat - w_B).abs().square().mean() * scale_recon
        
        
        loss_A = cycle_editing_loss(A, params_edit_A, flame, flametex)
        loss_B = cycle_editing_loss(B, params_edit_B, flame, flametex)
#         loss_A = cycle_editing_loss(A, params_edit_B, flame, flametex)
#         loss_B = cycle_editing_loss(B, params_edit_A, flame, flametex)
        loss_edit = loss_A['all_loss'] + loss_B['all_loss']
#         loss_edit = loss_A['all_loss']
        
    
        total_loss += loss_recon
        total_loss += loss_edit
        
        
        optim.zero_grad()
        total_loss.backward()
        optim.step()
        
        
    if epoch % 100 == 0:
        print("epoch: ", epoch, ", loss_edit: ", total_loss.item(), ", loss_recon: ", loss_recon.item())

epoch:  0 , loss_edit:  2.1838061809539795 , loss_recon:  0.0013645573053508997
epoch:  100 , loss_edit:  2.1868162155151367 , loss_recon:  0.0008672431576997042
epoch:  200 , loss_edit:  2.173971652984619 , loss_recon:  0.0018171978881582618
epoch:  300 , loss_edit:  2.2019596099853516 , loss_recon:  0.0010616485960781574
epoch:  400 , loss_edit:  2.199246883392334 , loss_recon:  0.0007042901124805212
epoch:  500 , loss_edit:  2.1893868446350098 , loss_recon:  0.00011368238483555615
epoch:  600 , loss_edit:  2.199608564376831 , loss_recon:  8.94882577995304e-06
epoch:  700 , loss_edit:  2.1680824756622314 , loss_recon:  2.39634368881525e-06
epoch:  800 , loss_edit:  2.2097766399383545 , loss_recon:  2.2328259774440085e-07
epoch:  900 , loss_edit:  2.176872968673706 , loss_recon:  5.433705609902972e-07
epoch:  1000 , loss_edit:  2.20977520942688 , loss_recon:  2.4079341187643877e-07
epoch:  1100 , loss_edit:  2.1632168292999268 , loss_recon:  1.0533121894695796e-06
epoch:  1200 , loss_

In [36]:
# from models.RigNet import StyleEncoder, StyleDecoder, RigNet
# import load

In [29]:
# dims_enc_in=(18,512)
# dims_enc_out=(18,32) 
# one_hot=True
# num_classes=6

# enc = StyleEncoder(dims_enc_in, dims_enc_out, one_hot, num_classes).cuda()

In [39]:
# dims_dec_in = (18, 268)
# dims_dec_out=(18, 512)

# if one_hot:
#     dims_dec_in = (dims_dec_in[0], dims_dec_in[-1] + num_classes)

# del dec
# dec = StyleDecoder(dims_dec_in, dims_dec_out).cuda()

In [30]:
# x = enc(w_B, labels_in)
# print(x.shape)

torch.Size([4, 18, 38])


In [40]:
# x = dec(x, params_A)

params_flat:  torch.Size([4, 18, 236])
w_enc:  torch.Size([4, 18, 38])
params_flat:  torch.Size([4, 18, 236])
x:  torch.Size([4, 18, 274])
dims_in:  (18, 274)


---

In [201]:
# rignet = load.rignet(base_path, one_hot=True).cuda()

Creating RigNet model...ONE_HOT


In [43]:
# labels_in = F.one_hot(torch.tensor([4]), 6).unsqueeze(0).expand(4,-1,-1)
# labels_in = labels_in.cuda()


# print(labels_in.shape)
# print(w_B.shape)

In [44]:
# w_Ahat = rignet(w_B, params_A, labels_in) # Transfer semantic params from A to latent B

In [45]:
# # 2) Transfer semantic "style" (e.g. pose) from one latent to another.
# #
# w_Ahat = rignet(w_B, params_A) # Transfer semantic params from A to latent B
# w_Bhat = rignet(w_A, params_B) # Transfer semantic params from B to latent A